# 🚗 AV Simulation on Google Colab - Setup Guide

This notebook walks you through setting up the Autonomous Vehicle Simulation system on Google Colab. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/av-simulation/blob/main/examples/notebooks/01_colab_setup.ipynb)

## 📋 What You'll Learn
- How to install the AV simulation on Colab
- Setting up the virtual display for GUI applications
- Running your first simulation
- Accessing simulation data

## ⚡ Prerequisites
- Google account for Colab access
- Basic Python knowledge
- Understanding of Jupyter notebooks

## 🔧 Step 1: Environment Setup

In [ ]:
# Install system dependencies for pygame and display
!apt-get update
!apt-get install -y xvfb python3-opengl ffmpeg

# Install Python packages
!pip install pygame pandas numpy matplotlib seaborn plotly scipy scikit-learn
!pip install pyvirtualdisplay pillow

print("✅ System dependencies installed successfully!")

## 📥 Step 2: Download AV Simulation Code

In [ ]:
# Option 1: Clone from GitHub (if available)
# !git clone https://github.com/YOUR_USERNAME/av-simulation.git
# %cd av-simulation

# Option 2: Upload manually or create the structure
import os
import sys

# Create project structure
!mkdir -p av_simulation/src/av_simulation/{core,data,detection,planning}
!mkdir -p av_simulation/examples/{notebooks,data,utils}
!mkdir -p av_simulation/simulation_data

print("📁 Project structure created!")
print("📝 Note: You'll need to upload the simulation files manually or clone from GitHub")

## 🖥️ Step 3: Setup Virtual Display

Since Colab doesn't have a display, we need to create a virtual one for pygame.

In [ ]:
from pyvirtualdisplay import Display
import pygame
import os

# Start virtual display
display = Display(visible=0, size=(1200, 800))
display.start()

# Set display environment variable
os.environ['DISPLAY'] = ':' + str(display.display)

# Test pygame initialization
try:
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    pygame.quit()
    print("✅ Virtual display setup successful!")
    print(f"🖥️  Display: {os.environ.get('DISPLAY')}")
except Exception as e:
    print(f"❌ Display setup failed: {e}")

## 📝 Step 4: Create Helper Functions for Colab

In [ ]:
import threading
import time
import io
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

class ColabSimulationRunner:
    """Helper class to run simulations in Colab environment"""
    
    def __init__(self):
        self.simulation = None
        self.running = False
        self.thread = None
        self.screenshots = []
        
    def capture_screenshot(self, screen):
        """Capture pygame screen as image"""
        w, h = screen.get_size()
        raw = pygame.image.tostring(screen, 'RGB')
        image = Image.frombytes('RGB', (w, h), raw)
        return image
    
    def run_headless_simulation(self, duration=30, capture_interval=2):
        """Run simulation without display for data collection"""
        from av_simulation.core.simulation import Simulation
        
        # Create simulation with data collection enabled
        sim = Simulation(enable_data_collection=True)
        
        start_time = time.time()
        last_capture = 0
        
        print(f"🚀 Starting {duration}s simulation...")
        
        while time.time() - start_time < duration:
            # Run one simulation step
            current_time = time.time() - start_time
            
            # Handle events (automated)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    break
            
            # Update simulation
            if sim.current_env.ego_vehicle:
                action = sim.planner.plan_action(sim.current_env.ego_vehicle)
                sim.current_env.ego_vehicle.set_action(action)
            
            sim.current_env.step(1.0/60)  # 60 FPS
            
            # Collect data
            if sim.data_collection_enabled:
                sim.collect_simulation_data()
            
            # Capture screenshot periodically
            if current_time - last_capture >= capture_interval:
                sim.current_env.draw(sim.screen)
                sim.draw_hud()
                pygame.display.flip()
                
                screenshot = self.capture_screenshot(sim.screen)
                self.screenshots.append((current_time, screenshot))
                last_capture = current_time
                
                print(f"📸 Captured screenshot at {current_time:.1f}s")
            
            time.sleep(1/60)  # Maintain 60 FPS
        
        # Save final simulation data
        if sim.data_collection_enabled and sim.current_run_id:
            sim.data_repository.end_current_run()
            print(f"💾 Simulation data saved: {sim.current_run_id}")
        
        pygame.quit()
        return sim.data_repository if sim.data_collection_enabled else None
    
    def show_screenshots(self, max_images=6):
        """Display captured screenshots"""
        if not self.screenshots:
            print("No screenshots captured")
            return
        
        n_images = min(len(self.screenshots), max_images)
        cols = 3
        rows = (n_images + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
        if rows == 1:
            axes = [axes] if cols == 1 else axes
        else:
            axes = axes.flatten()
        
        for i in range(n_images):
            timestamp, image = self.screenshots[i]
            axes[i].imshow(image)
            axes[i].set_title(f"t = {timestamp:.1f}s")
            axes[i].axis('off')
        
        # Hide empty subplots
        for i in range(n_images, len(axes)):
            axes[i].axis('off')
        
        plt.tight_layout()
        plt.show()

# Create global runner instance
colab_runner = ColabSimulationRunner()

print("✅ Colab helper functions created!")

## 🎮 Step 5: Test Basic Pygame Functionality

In [ ]:
import pygame
import matplotlib.pyplot as plt
from PIL import Image

# Test pygame with virtual display
pygame.init()
screen = pygame.display.set_mode((400, 300))
pygame.display.set_caption("AV Simulation Test")

# Draw a simple test pattern
screen.fill((50, 50, 50))  # Dark gray background
pygame.draw.circle(screen, (0, 255, 0), (100, 150), 30)  # Green circle (ego vehicle)
pygame.draw.circle(screen, (0, 100, 255), (300, 150), 25)  # Blue circle (other vehicle)
pygame.draw.rect(screen, (255, 255, 0), (50, 100, 300, 10))  # Yellow road marking

pygame.display.flip()

# Capture and display the test image
w, h = screen.get_size()
raw = pygame.image.tostring(screen, 'RGB')
image = Image.frombytes('RGB', (w, h), raw)

pygame.quit()

# Display the captured image
plt.figure(figsize=(8, 6))
plt.imshow(image)
plt.title("🎮 Pygame Test - Virtual Display Working!")
plt.axis('off')
plt.show()

print("✅ Pygame test successful! Ready for AV simulation.")

## 📁 Step 6: Create Simulation Files Structure

**Note:** In a real setup, you would upload your simulation files or clone from GitHub. For this demo, we'll create placeholder files.

In [ ]:
# Create necessary __init__.py files
init_files = [
    'av_simulation/src/av_simulation/__init__.py',
    'av_simulation/src/av_simulation/core/__init__.py',
    'av_simulation/src/av_simulation/data/__init__.py',
    'av_simulation/src/av_simulation/detection/__init__.py',
    'av_simulation/src/av_simulation/planning/__init__.py'
]

for init_file in init_files:
    os.makedirs(os.path.dirname(init_file), exist_ok=True)
    with open(init_file, 'w') as f:
        f.write('# Package initialization\n')

# Add to Python path
import sys
sys.path.insert(0, '/content/av_simulation/src')

print("📁 File structure created!")
print("📝 Upload your simulation files to the av_simulation/ directory")
print("🔗 Or clone from GitHub: git clone https://github.com/YOUR_USERNAME/av-simulation.git")

## 🚀 Step 7: Quick Simulation Test

Once you have uploaded the simulation files, run this cell to test the system:

In [ ]:
# This cell assumes you have uploaded the complete simulation code

try:
    # Test import
    from av_simulation.data.repository import DataRepository
    from av_simulation.data.exporters import CSVExporter, JSONExporter
    
    # Create test repository
    repo = DataRepository("/content/test_simulation_data")
    
    print("✅ AV Simulation imports successful!")
    print("🚗 Ready to run simulations!")
    
    # Show next steps
    print("\n📋 Next Steps:")
    print("1. Open '02_simulation_runner.ipynb' to run simulations")
    print("2. Open '03_data_analysis.ipynb' to analyze results")
    print("3. Open '04_visualization_examples.ipynb' for plotting")
    
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("📁 Please upload the simulation files first")
    print("🔗 Or follow the GitHub cloning instructions above")

## 🔧 Troubleshooting

### Common Issues:

**Display Issues:**
- Restart runtime and re-run the display setup
- Check if Xvfb is properly installed

**Import Errors:**
- Ensure all simulation files are uploaded
- Check Python path configuration
- Verify file structure matches expected layout

**Performance Issues:**
- Use shorter simulation durations
- Reduce screenshot capture frequency
- Consider using Colab Pro for better resources

### Getting Help:
- Check the main project documentation
- Review the example notebooks
- Open an issue on GitHub

## 🎉 Conclusion

You've successfully set up the AV Simulation environment on Google Colab! 

### What's Ready:
✅ Virtual display for GUI applications  
✅ Python environment with all dependencies  
✅ Project structure and helper functions  
✅ Screenshot capture capabilities  
✅ Data collection system  

### Next Steps:
1. **Upload Simulation Files** - Clone from GitHub or upload manually
2. **Run Simulations** - Use the next notebook to start collecting data
3. **Analyze Results** - Explore the data analysis notebooks
4. **Create Visualizations** - Generate plots and insights

---

**Happy Simulating! 🚗💨**

Move on to: **[02_simulation_runner.ipynb](02_simulation_runner.ipynb)**